# Text -> Tokenized text --> Token Ids --> Token Embedding --> Positional Embedding (Absolute or Relative).

### Token Embedding + Positional Embedding = Input for LLM embedding

In [11]:
! pip3 install tiktoken
! pip3 install striprtf

In [13]:
from striprtf.striprtf import rtf_to_text

with open("/text.rtf", "r", encoding="utf-8") as f:
    raw_text = f.read()

raw_text = rtf_to_text(raw_text)

print("Total number of characters (clean): ", len(raw_text))
print("--- Preview ---")
print(raw_text[:99])

Total number of characters (clean):  2982
--- Preview ---
Personal Statement
Engineering the best solutions to complex problems is rarely a matter of raw com


**Positional Embedding**

We will use much larger vocab size for realistic view.

In [20]:
import torch

vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [21]:
from torch.utils.data import Dataset, DataLoader
import tiktoken

class GPTDatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    # tokenize the entire text
    token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

    # use sliding window with max_length as sequence length
    # since we will be sliding by 4 words, the stride here will be 4
    # which means we skip 4 words to create next sequence.
    for i in range(0, len(token_ids)-max_length, stride):
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

In [22]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

  # Initialize the tokenizer
  tokenizer = tiktoken.get_encoding("gpt2")

  # Initialize the dataset
  dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

  # Create the dataloader
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle,
                          drop_last=drop_last, num_workers=num_workers)

  return dataloader

In [23]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
    )

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [24]:
print("Token IDs:\n", inputs)
print("\nInput Shapes:\n", inputs.shape)

Token IDs:
 tensor([[30228, 21983,   198, 13798],
        [ 1586,   262,  1266,  8136],
        [  284,  3716,  2761,   318],
        [ 8365,   257,  2300,   286],
        [ 8246, 14492,  1176,   198],
        [17749,    26,   340,   318],
        [ 8793,   416,  8263,   422],
        [  262, 10084, 45207,   286]])

Input Shapes:
 torch.Size([8, 4])


In [25]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


**Context length can be used as max length as for a single batch we will be using the position and for other batches we will continue with same positional adding.**

In [29]:
context_length = max_length # which is first batch = 4
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
# same as token embedding we create random postion embedding and later optimize during LLM training.

In [30]:
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


Add the pos_embeddings to token embedding

In [31]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
